In [1]:
import os
import pandas as pd
import numpy as np
import openmatrix as omx
import random
import yaml

from utility import *

import warnings
warnings.filterwarnings('ignore')

In [2]:
with open('config.yaml', 'r') as file:
    params = yaml.safe_load(file)
    
_join = os.path.join
_dir = os.path.dirname
_norm = os.path.normpath

# paths
model_outputs_dir = params['model_dir']
skims_dir = _join(model_outputs_dir, "skims")
summary_outputs = params['summary_dir']
concept_id = params['concept_id']
ctramp_dir = params['ctramp_dir']
iteration = params['iteration']

concept_id = params['concept_id']
time_period_mapping = params['time_periods_mapping']
link21_purp_mapping = params['purpose_mapping']
mode_cat_mapping = params['mode_mapping']
time_periods = params['periods']
acc_egg_modes = params['access_egress_modes']


highway_output_data = params['highway_output_data']

In [3]:
# outputs of CT-RAMP model for tour and trip file
household_model_dir = _join(model_outputs_dir, "main")

# input household and person data
person_file = _join(ctramp_dir, 'main\\personData_' + str(iteration) + '.csv')
household_file = _join(ctramp_dir, 'main\\householdData_' + str(iteration) + '.csv')

person = pd.read_csv(person_file)

hh = pd.read_csv(household_file, usecols = ['hh_id', 'taz'])
hh = hh.rename(columns = {'taz': 'home_zone'})

#taz to RDM zones, super districts, county
geo_cwks = pd.read_csv(_join(params['common_dir'], "geographies.csv")) #columns taz, rdm_zones, super_district, county

#taz to priority population
pp_perc = pd.read_excel(_join(params['common_dir'], "TAZ_Tract_cwk_summary.xlsx")) #columns = taz, pp_share 

# transbay od pairs
transbay_od = pd.read_csv(_join(params['common_dir'], "transbay_od.csv")) #columns = transbay_o, transbay_d

demand_matrices_dir = _join(model_outputs_dir, "demand_matrices")
transit_demand_dir = _join(demand_matrices_dir, "transit")
transit_skims_dir = _join(skims_dir, "transit")
best_path_skim_dir = params['best_path_skim_dir']

perf_measure_columns = params['final_columns']

period = params['periods']

In [10]:
period

['am', 'md', 'pm', 'ev', 'ea']

In [6]:
am = pd.read_csv(_join(highway_output_data, 'Volumes and times (on links)_am.csv'), sep='\t')

In [13]:
df_vmt = pd.DataFrame(columns=['Period', 'Value'])
for prd in period:
    tod = pd.read_csv(_join(highway_output_data, f'Volumes and times (on links)_{prd}.csv'), sep='\t')    
    df_vmt = df_vmt.append({'Period': prd, 'Value': tod['VDT'].sum()}, ignore_index=True)

In [14]:
df_vmt

,Period,Value
0,am,4.262354e+07
1,md,5.109353e+07
2,pm,4.506160e+07
3,ev,2.804407e+07
4,ea,7.296842e+06


In [15]:
df_vmt['Concept_ID'] = concept_id
df_vmt['Metric_ID'] = 'A3.4'
df_vmt['Metric_name'] = ''
df_vmt['Submetric'] = 'A3.4.1'
df_vmt['Description'] = 'VMT reduction (network)'
df_vmt['Population'] = 'Whole Population'
df_vmt['Geography'] = 'Regional'
df_vmt['Zone_ID'] = ''
df_vmt['Origin_zone'] = ''
df_vmt['Dest_zone'] = ''
df_vmt['Purpose'] = ''
df_vmt['Units'] = 'vehicle miles travelled'
df_vmt['Total_Increment'] = ''